In [ ]:
!pip install xarray

In [ ]:
import xarray as xr
import numpy as np


data = xr.DataArray(np.random.randn(2, 3), dims=("x", "y"), coords={"x": [10, 100]})

ds = xr.Dataset(dict(foo=data, bar=("x", [1, 2]), baz=np.pi))
ds

In [ ]:
# This is the "secret sauce". We create a class which is a subclass of xarray.Index
# https://docs.xarray.dev/en/latest/internals/how-to-create-custom-index.html

from xarray.core.indexes import PandasIndex, IndexSelResult, normalize_label, get_indexer_nd

# However, the built-in PandasIndex is _almost_ what we want,
# so we subclass it to steal most of the functionality
class LogIndexer(PandasIndex):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  # Except for .sel, where we can inject our custom logic!
  def sel(self, labels: dict, method=None, tolerance=None) -> IndexSelResult:
    # based on the code in https://github.com/pydata/xarray/blob/ce5130f39d780cdce87366ee657665f4a5d3051d/xarray/core/indexes.py#L745
    if method == "nearest_log":
      assert len(labels) == 1
      coord_name, label = next(iter(labels.items()))
      label_array = normalize_label(label, dtype=self.coord_dtype)
      indexer = get_indexer_nd(np.log(self.index), np.log(label_array), method="nearest", tolerance=tolerance)
      return IndexSelResult({self.dim: indexer})
    return super().sel(labels, method, tolerance)

In [ ]:
# Now we need to make a dataset that uses this custom index
# this odd incantation is the best I've found, but there may be others.
ds_log = ds.drop_indexes('x').set_xindex('x', LogIndexer)
ds_log

In [ ]:
# x has values 10 and 100, to make it easy to observe the differences here:

print("normal:\t", ds_log.sel(x=[10,20,30,40,50,60], method="nearest").x.data)
print("log:\t", ds_log.sel(x=[10,20,30,40,50,60], method="nearest_log").x.data)